<a href="https://colab.research.google.com/github/HanaIstfalen/exam_2025/blob/main/notebooks/exercices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [2]:
! git clone https://github.com/HanaIstfalen/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 12.87 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [3]:
mySeed = 200

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [4]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

In [6]:
# Une méthode simple pour estimer les coefficients θk est la
# méthode des moindres carrés ordinaires
X = train_set['inputs']
y = train_set['targets']

X = np.c_[np.ones(X.shape[0]), X]

theta = np.linalg.solve(X.T @ X, X.T @ y)

print("Coefficients estimés (θ) :", theta)

Coefficients estimés (θ) : [10.07876403  1.95156862  1.94842221  3.59966699]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

In [9]:
# Architecture : Un réseau de neurones monocouche avec une fonction d'activation linéaire.
# Expressivité : Ce modèle correspond à un perceptron multicouche réduit à une seule couche (SLP).
# Avec une fonction d'activation linéaire, il capture parfaitement la relation t = θ0 + θ1x + θ2y + θ3z + ε,
# car cette relation est une combinaison linéaire des prédicteurs.
# Généralisation : Ce modèle est simple et efficace pour cette tâche car il utilise exactement 4 paramètres (θ0, θ1, θ2, θ3),
# ce qui est suffisant pour estimer une relation linéaire sans surajuster.


In [10]:
import torch.nn as nn
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [11]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:

      optimizer.zero_grad()
      outputs = mySimpleNet(batch_inputs)
      loss = criterion(outputs, batch_targets.unsqueeze(1))
      loss.backward()
      optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [50/500], Loss: 4.3438
Epoch [100/500], Loss: 4.6993
Epoch [150/500], Loss: 4.3906
Epoch [200/500], Loss: 3.5670
Epoch [250/500], Loss: 3.5301
Epoch [300/500], Loss: 3.6811
Epoch [350/500], Loss: 3.6629
Epoch [400/500], Loss: 3.4460
Epoch [450/500], Loss: 4.6915
Epoch [500/500], Loss: 5.8766


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [12]:
# Les estimations des θk sont stockées dans les paramètres de la couche linéaire (self.linear) du modèle mySimpleNet.
# Extraction des poids et du biais
poids = mySimpleNet.linear.weight.data
biais = mySimpleNet.linear.bias.data

print("θ0 estimé (biais) :", biais.item())
print("θ1, θ2, θ3 estimés (poids) :", poids.tolist())

θ0 estimé (biais) : 10.07835865020752
θ1, θ2, θ3 estimés (poids) : [[1.949466347694397, 1.948156476020813, 3.5991649627685547]]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [16]:
import torch

# --- Données de test ---
X_test = test_set['inputs']
y_test = test_set['targets']

# --- Coefficients MCO (de la question 1) ---
theta_ols = [10.07876403, 1.95156862, 1.94842221, 3.59966699]

# --- Prédictions avec le réseau de neurones ---
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
with torch.no_grad():
    predictions_nn = mySimpleNet(X_test_tensor)
predictions_nn = predictions_nn.numpy().squeeze()

# --- Prédictions avec les MCO ---
X_test_ols = np.c_[np.ones(X_test.shape[0]), X_test]
predictions_ols = X_test_ols @ theta_ols

# --- Calcul des MSE ---
mse_nn = np.mean((predictions_nn - y_test)**2)  # MSE pour le réseau de neurones
mse_ols = np.mean((predictions_ols - y_test)**2) # MSE pour les MCO

# --- Affichage des résultats ---
print("MSE pour les MCO :", mse_ols)
print("MSE pour le réseau de neurones :", mse_nn)

MSE pour les MCO : 4.009260943786592
MSE pour le réseau de neurones : 4.009683638890263


Les résultats des deux méthodes d'estimation sont très proches. Cela indique que le réseau de neurones a réussi à apprendre la relation linéaire entre les prédicteurs et la cible de manière efficace.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [17]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Le réseau de neuronnes présenté est un Fully Convolutional Network (FCN), plus précisément une variante causale de celui-ci.

Nombre de paramètres dans self.conv1: out_ch * (in_ch * kernel_size + 1)

Nombre de paramètres dans self.conv1: out_ch * (out_ch * kernel_size + 1)

Nombre de paramètres dans self.bn1: 2 * out_ch

Nombre de paramètres dans self.bn2: 2 * out_ch

on a size = 64, kernel_size = 3, in_ch_conv1 = 1, out_ch_conv1 = in_ch_conv2 = 128, out_ch_conv2 = 128

Ainsi, le nombre de paramètres de la couche self.Down1 est la somme de tous ces paramètres : 50432

In [18]:
# Nb de paramètres au total:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = causalFCN()
total_params = count_parameters(model)
print(f"Nombre total de paramètres du modèle : {total_params}")

Nombre total de paramètres du modèle : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

Mécanismes de réduction de la taille : Convolutions causales sans padding et MaxPooling (down3)

Mécanismes de restitution de la taille : UpSampling (up2) et Concaténation (up2, up3, up4)

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Mécanismes d'augmentation du champ réceptif:

* Convolutions empilées: Chaque couche convolutive augmente le champ réceptif, permettant au réseau de "voir" une portion plus large de l'entrée.
* Dilatations: Les dilatations augmentent l'espacement entre les éléments du noyau de convolution, élargissant le champ réceptif sans augmenter le nombre de paramètres.

Taille du champ réceptif en sortie de self.inc : self.inc est composé de deux couches convolutives causales avec kernel_size=3 et dilation=1.
Chaque couche augmente le champ réceptif de kernel_size - 1 = 2.
Donc, le champ réceptif en sortie de self.inc est de 2 + 2 = 4.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [19]:
input_tensor1 = torch.zeros(1, 1, 10000)
input_tensor2 = input_tensor1.clone()
input_tensor2[0, 0, 0] = 1  # Modifier une seule composante

with torch.no_grad():
    output1 = model(input_tensor1)
    output2 = model(input_tensor2)

diff = output2 - output1

receptive_field_indices = torch.nonzero(diff[0, 0]).squeeze()

receptive_field_size = receptive_field_indices[-1].item() + 1

print("Taille du champ réceptif pour y5000 :", receptive_field_size)

Taille du champ réceptif pour y5000 : 10000


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



Non, y5000 ne dépend pas des composantes x_t pour t > 5000.

Justification empirique : e champ réceptif pour y5000 est de 10000, ce qui signifie qu'il couvre l'ensemble de la séquence d'entrée de x0 à x9999. Cependant, le modèle est causal, ce qui implique que la sortie à un instant donné ne dépend que des entrées passées et présentes. Par conséquent, y5000 ne peut pas dépendre des composantes x_t pour t > 5000, car ces composantes représentent des entrées futures par rapport à l'instant 5000.

La partie du code garantissant la causalité dans Double_conv_causal est le padding :

x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))

Ce padding ajoute des zéros uniquement du côté gauche de la séquence d'entrée, ce qui assure que la convolution ne prend en compte que les valeurs passées et présentes de l'entrée pour calculer la sortie.

\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?